In [74]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [75]:
m = gp.Model()

In [76]:
combos, profit = gp.multidict(
    {
        ("Device1", "Company1"): 4,
        ("Device2", "Company1"): 6,
        ("Device3", "Company1"): 7,
        ("Device4", "Company1"): 7,
        ("Device5", "Company1"): 7,
        ("Device6", "Company1"): 7,
        ("Device1", "Company2"): 2,
        ("Device2", "Company2"): 4,
        ("Device3", "Company2"): 6,
        ("Device4", "Company2"): 8,
        ("Device5", "Company2"): 9,
        ("Device6", "Company2"): 10,
        ("Device1", "Company3"): 3,
        ("Device2", "Company3"): 5,
        ("Device3", "Company3"): 7,
        ("Device4", "Company3"): 8,
        ("Device5", "Company3"): 8,
        ("Device6", "Company3"): 8,
        ("Device1", "Company4"): 4,
        ("Device2", "Company4"): 5,
        ("Device3", "Company4"): 6,
        ("Device4", "Company4"): 6,
        ("Device5", "Company4"): 6,
        ("Device6", "Company4"): 6,
    }
)

In [77]:
devices = set(combo[0] for combo in combos)
companies = set(combo[1] for combo in combos)

In [78]:
x = m.addVars(combos, vtype=GRB.BINARY, name="x")

In [79]:
m.setObjective(x.prod(profit), sense=GRB.MAXIMIZE)

In [80]:
m.addConstrs((x.sum(device, "*") == 1 for device in devices), name="Device只能给一家")

{'Device2': <gurobi.Constr *Awaiting Model Update*>,
 'Device5': <gurobi.Constr *Awaiting Model Update*>,
 'Device6': <gurobi.Constr *Awaiting Model Update*>,
 'Device3': <gurobi.Constr *Awaiting Model Update*>,
 'Device4': <gurobi.Constr *Awaiting Model Update*>,
 'Device1': <gurobi.Constr *Awaiting Model Update*>}

In [81]:
m.addConstrs(
    (x.sum("*", company) >= 1 for company in companies), name="企业至少分得一台Device"
)

{'Company1': <gurobi.Constr *Awaiting Model Update*>,
 'Company4': <gurobi.Constr *Awaiting Model Update*>,
 'Company3': <gurobi.Constr *Awaiting Model Update*>,
 'Company2': <gurobi.Constr *Awaiting Model Update*>}

In [82]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 24 columns and 48 nonzeros
Model fingerprint: 0xb36c0d1d
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 39.0000000
Presolve time: 0.00s
Presolved: 10 rows, 24 columns, 48 nonzeros
Variable types: 0 continuous, 24 integer (24 binary)
Found heuristic solution: objective 43.0000000

Root relaxation: objective 4.400000e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      44.0000000   44.00000  0.00

In [83]:
m.write("IP.lp")

In [84]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 44.0
Optimal solution:
x[Device2,Company1] = 1.0
x[Device4,Company2] = 1.0
x[Device5,Company2] = 1.0
x[Device6,Company2] = 1.0
x[Device3,Company3] = 1.0
x[Device1,Company4] = 1.0
